In [1]:
import sys,os
print(sys.version)
print(os.getcwd())

3.5.2 |Anaconda 4.2.0 (x86_64)| (default, Jul  2 2016, 17:52:12) 
[GCC 4.2.1 Compatible Apple LLVM 4.2 (clang-425.0.28)]
/Users/zishuoli/Downloads/bigdata/final


In [2]:
'''import basic package'''
import pandas as pd
import numpy as np


'''import deep learning package'''
import keras as k
import tensorflow as tf


import requests
import json

from IPython.display import Image
from IPython.display import display
from IPython.display import HTML


Using TensorFlow backend.


In [3]:
rating_df = pd.read_csv('ml-latest-small/ratings.csv')
linkes_df = pd.read_csv('ml-latest-small/links.csv')

df_merged = pd.merge(rating_df,linkes_df,on=['movieId'])
print(df_merged.head())
print(df_merged.shape)

   userId  movieId  rating   timestamp  imdbId  tmdbId
0       1       31     2.5  1260759144  112792  9909.0
1       7       31     3.0   851868750  112792  9909.0
2      31       31     4.0  1273541953  112792  9909.0
3      32       31     4.0   834828440  112792  9909.0
4      36       31     3.0   847057202  112792  9909.0
(100004, 6)


In [4]:
rating_matrix_dim = [len(df_merged.userId.unique()),max(df_merged.movieId)]

rating_matrix = np.zeros(rating_matrix_dim)


for row in df_merged.itertuples():
    try:
        rating_matrix[row[1]-1, row[2]-1] = row[3]
    except:
        continue
rating_matrix = rating_matrix[:,:9000]

print(rating_matrix.shape)

(671, 9000)


In [5]:
'''
split train & test
'''
train_matrix = rating_matrix.copy()
test_matrix = np.zeros(rating_matrix.shape)

for i in range(rating_matrix.shape[0]):
    rating_idx = np.random.choice(
        rating_matrix[i, :].nonzero()[0], 
        size=10, 
        replace=True)
    train_matrix[i, rating_idx] = 0.0
    test_matrix[i, rating_idx] = rating_matrix[i, rating_idx]


In [6]:
def cosine_similarity_matrix(data):
    '''
    calculate cosine similarity for user & item
    
    default: user is in row, item in column
    ''' 
    dim = data.shape
    
    constant = tf.constant(1e-9,dtype=tf.float32)

    df = tf.placeholder(shape=[dim[0],dim[1]],dtype=tf.float32)

    similar_user = tf.matmul(df,tf.transpose(df)) + constant

    norm_user = tf.reshape(tf.sqrt(tf.diag_part(similar_user)),[-1,1])

    norm_user_matrix = tf.matmul(norm_user,tf.transpose(norm_user))

    similar_user = similar_user/norm_user_matrix


    with tf.Session() as sess:
        ans = sess.run(similar_user,feed_dict={df:data})

    return ans


In [7]:
similarity_user = cosine_similarity_matrix(train_matrix)
print('similarity user matrix shape:',similarity_user.shape)
similarity_movie = cosine_similarity_matrix(train_matrix.T)
print('similarity movie matrix shape:',similarity_movie.shape)


similarity user matrix shape: (671, 671)
similarity movie matrix shape: (9000, 9000)


In [8]:
def prediction_(similarity_matrix,rating_matrix):
    '''default we think df row is user, column is id'''
    similarity_matrix = tf.convert_to_tensor(similarity_matrix,dtype=tf.float32)
    rating_matrix = tf.convert_to_tensor(rating_matrix,dtype=tf.float32)
    
    numerator = tf.matmul(similarity_matrix,rating_matrix)
    
    denominator = tf.reduce_sum(tf.abs(similarity_matrix),axis=0)
    denominator = tf.reshape(denominator,shape=[-1,1])

    ans = numerator/denominator
    
    with tf.Session() as sess:
        return sess.run(ans)
    

def error(prediction,real_value):
    
    prediction = tf.convert_to_tensor(prediction)
    real_value = tf.convert_to_tensor(real_value)
    
    ans = tf.losses.mean_squared_error(predictions=prediction,labels=real_value)
    with tf.Session() as sess:
        return sess.run(ans)

In [9]:
prediction = prediction_(similarity_user,rating_matrix)
prediction = prediction[test_matrix.nonzero()].flatten()
test_vector = test_matrix[test_matrix.nonzero()].flatten()
print('real_value:',test_vector)
print('prediction:',prediction)

real_value: [ 2.   4.   2.  ...,  5.   3.   4.5]
prediction: [ 0.9263624   0.81012529  0.79657292 ...,  1.98036313  0.4275431
  0.44862014]


In [10]:
mse = error(prediction=prediction,real_value=test_vector)
print('mse for prediction using cosine similarity',mse)

mse for prediction using cosine similarity 8.98062


In [12]:
api_key = 'api_key=b1ab9a6e9622fe27f6dd2a81065224fc'

def get_poster(TMDB_id,api_key=api_key):
    
    url_full = 'https://api.themoviedb.org/3/movie/'+ str(TMDB_id)+'?'+str(api_key)
    url_content = requests.get(url_full)
    url_content = url_content.text
    post_ip = json.loads(url_content)['poster_path']
    post_ip_complete = 'https://image.tmdb.org/t/p/w500'+ post_ip
    
    return post_ip_complete

In [26]:
'''built a map between movieId and tmdbID'''
movieId_vs_tmdbId = {}

for row in df_merged[['movieId','tmdbId']].itertuples():    
    try:
        movieId_vs_tmdbId[str(int(row[1]-1))] = int(row[2])
    except:
        continue

In [86]:
def similar_select(similarity_matrix,movieId_vs_tmdbId='',n=5,target_id=0):

    movies = [ movieId_vs_tmdbId[str(x)] \
          for x in np.argsort(similarity_matrix[target_id,:])[:-n-1:-1] ] 

    return movies

def display_images(url_list):
    n_display = len(url_list)
    images = ''
    for i in range(n_display):
        images += "<img style='width: 100px; margin: 0px; \
                float: left; border: 1px solid black;' src='%s' />" \
                % url_list[i]

    display(HTML(images)) 



In [17]:
movie_list = similar_select(similarity_movie,movieId_vs_tmdbId)
url_list = [get_poster(x) for x in movie_list]

display_images(url_list)

# Deep Learning Part

In [18]:

import urllib.request


from keras.applications import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image as kimage


In [27]:
'''this cell just run one time to download all posters'''
# poster_path = "/Users/zishuoli/Downloads/bigdata/final/posters/"

movie_list = []
# item in movie list is tmdb id
for i in movieId_vs_tmdbId.values():
    try:
        poster_url = get_poster(i)
        # download poster jpg to poster_path
        urllib.request.urlretrieve(poster_url, poster_path + str(int(i)) + ".jpg")
        movie_list.append(i)
    except:
        print('tmdb movie id :',str(i),'is not available')
        continue


tmdb movie id : 58423 is not available
tmdb movie id : 110414 is not available
tmdb movie id : 370646 is not available
tmdb movie id : 156078 is not available
tmdb movie id : 17882 is not available
tmdb movie id : 374454 is not available
tmdb movie id : 253941 is not available
tmdb movie id : 13057 is not available
tmdb movie id : 205300 is not available
tmdb movie id : 244797 is not available
tmdb movie id : 61920 is not available
tmdb movie id : 68149 is not available
tmdb movie id : 53285 is not available
tmdb movie id : 206216 is not available
tmdb movie id : 314040 is not available
tmdb movie id : 13716 is not available
tmdb movie id : 53803 is not available
tmdb movie id : 110639 is not available
tmdb movie id : 191903 is not available
tmdb movie id : 147538 is not available
tmdb movie id : 2518 is not available
tmdb movie id : 253768 is not available
tmdb movie id : 416437 is not available
tmdb movie id : 12773 is not available
tmdb movie id : 67479 is not available
tmdb movie i

In [29]:

'''transfer learning'''
img = []

poster_path = "/Users/zishuoli/Downloads/bigdata/final/posters/"

for i in movie_list:
    tmp = kimage.load_img(poster_path + str(i) + ".jpg", target_size=(224, 224))
    tmp = kimage.img_to_array(tmp)
    tmp = np.expand_dims(tmp, axis=0)
    tmp = preprocess_input(tmp)   
    
    img.append(tmp)
    
# image_top=False removes final connected layers   
model = VGG16(include_top=False, weights='imagenet')

In [34]:
movie_num = len(movie_list)

feature_matrix = np.zeros([movie_num,25088])

for i in range(movie_num):
    tmp = model.predict(img[i]).ravel()
    feature_matrix[i,:] = tmp

In [48]:
similarity_movie = cosine_similarity_matrix(feature_matrix)

In [88]:

def show_result(similarity_matrix,target_id = 0,n = 5):
        
    movie_selected_id = [x for x in np.argsort(similarity_movie[target_id,:])[-n-1:]]

    movie_selected = [movie_list[i] for i in movie_selected_id]

    url_list = [get_poster(x) for x in movie_selected]
    get_poster(movie_selected[0])
    display_images(url_list)

In [94]:
show_result(similarity_movie,7)
